# 安裝 fugle-trade 套件

In [1]:
# pip install 需要的套件
!pip install fugle-trade==0.1.8

     |████████████████████████████████| 3.9 MB 6.9 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 2.0 MB 27.5 MB/s 
     |████████████████████████████████| 3.6 MB 23.6 MB/s 
  Created wheel for keyrings.cryptfile: filename=keyrings.cryptfile-1.3.8-py2.py3-none-any.whl size=17837 sha256=1631740de48823a92d55b77adc38e41167dac0cce26e11362930c81068af8743
  Stored in directory: /root/.cache/pip/wheels/4d/27/4d/c5d8637dc5fd0fac41b838ca49f0bbbde7b6cf47b8fa4339da
Successfully built keyrings.cryptfile


In [2]:
from configparser import ConfigParser
from fugle_trade.sdk import SDK

from fugle_trade.order import OrderObject
from fugle_trade.constant import (APCode, Trade, PriceFlag, BSFlag, Action)

# 模擬環境測試

您在首次使用交易 API 取得正式環境的使用權限前，您會需要在模擬環境至少送出一筆委託

In [ ]:
## 取得 new 模擬環境設定檔並登入

config = ConfigParser()

config.read('./config.simulation.ini')

sdk = SDK(config)

sdk.login()

Enter esun account password:
··········
Please set a password for your new keyring: ··········
Please confirm the password: ··········
Enter cert password:
··········


In [ ]:
t## 模擬金鑰下單測試
order = OrderObject(
    buySell = Action.Buy,
    price = 27.0,
    stockNo = "2884",
    quantity = 2,
)

sdk.place_order(order)

print("Your order has been placed successfully.")

In [ ]:
## 查看委託回報
sdk.get_order_results()

In [ ]:
## 更改密碼
# sdk.reset_password()

# 正式環境

在 colab 環境因安全性考量，會需要增加 keyring 來設置密碼，這邊的第一個密碼是證券登入密碼，第四個則是憑證密碼，中間兩個則是相同的 keyring 密碼

In [4]:
# 取得正式環境設定檔並登入

config = ConfigParser()

config.read('./config.ini')

sdk = SDK(config)

sdk.login()

Enter esun account password:
··········
Please set a password for your new keyring: ··········
Please confirm the password: ··········
Enter cert password:
··········


## 下委託

In [5]:
# 下委託單
order = OrderObject(
    apCode = APCode.Common,# 盤別
    buySell = Action.Buy,
    priceFlag= PriceFlag.LimitDown, # 下跌停價買進
    price = None,
    stockNo = str(2884),
    quantity = 10,
)

sdk.place_order(order)

{'ordDate': '20220315',
 'ordTime': '162644694',
 'ordType': '1',
 'ordno': 'P0D85NHS',
 'retcode': '000000',
 'retmsg': '',
 'workDate': '20220316'}

In [6]:
# 委託回報
orderResult = sdk.get_order_results()

# 查看委託回報狀況
import pandas as pd
pd.DataFrame(orderResult)

,market,workdate,orddate,ordtime,syscode,ordstatus,ordno,preordno,stockno,buysell,...,celable,errcode,errmsg,avgPrice,chgtime,chgdate,bsFlag,orgqty_share,matqty_share,celqty_share
0,T,20220316,20220315,162644694,FA,1,,P0D85NHS,2884,B,...,1,00000000,,0,162644694,20220315,R,10000,0,0
1,T,20220316,20220315,145246203,FA,1,,P0D85N9F,2884,B,...,2,00000000,,0,145303275,20220315,R,10000,0,10000
2,NaN,20220315,20220315,132115772,FA,2,C1141,,2884,B,...,2,00000000,,0.0,132222419,20220315,R,10000,0,10000
3,NaN,20220315,20220315,132102194,FA,2,C1115,,2884,B,...,2,00000000,,0.0,132226438,20220315,R,10000,0,10000
4,NaN,20220315,20220315,112715893,FA,2,B3927,,2884,B,...,2,00000000,,0.0,113012408,20220315,R,10000,0,10000
5,NaN,20220315,20220315,105502398,FA,2,B1824,,2884,B,...,2,00000000,,0.0,113022522,20220315,R,10000,0,10000
6,NaN,20220315,20220315,105340069,FA,2,B1725,,2884,B,...,2,00000000,,0.0,105445117,20220315,R,10000,0,10000
7,NaN,20220315,20220315,103856801,FA,2,B0624,,2884,B,...,2,00000000,,0.0,105440112,20220315,R,10000,0,10000
8,NaN,20220315,20220315,103741864,FA,2,B0534,,2884,B,...,2,00000000,,0.0,103824099,20220315,R,10000,0,10000
9,NaN,20220315,20220315,103658933,FA,2,B0453,,2884,B,...,2,00000000,,0.0,103829772,20220315,R,10000,0,10000


In [7]:
# 刪掉最新一筆委託單
sdk.cancel_order(orderResult[0])

{'ordDate': '20220315',
 'ordTime': '162701463',
 'retcode': '000000',
 'retmsg': ''}

# 簡易策略 demo code

In [8]:
# 安裝相關套件
import requests
import numpy as np
import pandas as pd
import datetime
import time

In [9]:
# 取得正式環境設定檔並登入

config = ConfigParser()

config.read('./config.ini')

sdk = SDK(config)

sdk.login()

In [10]:
# 將 行情 API 自訂 function 方便後續使用

def fugle_realtime_api(apiType, symbolId, apiToken):

    # 連結 fugle realtime API 取得最新數據 
    api_path = f"https://api.fugle.tw/realtime/v0.3/intraday/{apiType}?symbolId={symbolId}&apiToken={apiToken}"

    price_col = requests.get(api_path)

    price_data = price_col.json()['data'][apiType]

    return price_data


## 觸量下單 demo



策略描述：<br>
盤中突然的大單，可以反映大戶的交易意願提升，而外盤成交代表買方願意用比較高的價格買，代表有較強烈的買進意圖<br>
因此這裡建構一個簡單的策略：當最新一筆成交是外盤成交，且成交量是前一筆的 10 倍以上就跟！

In [11]:
## set parameter

# 輸入股票代碼
symbolId = "2884"

# 請輸入自己的 apiToken
apiToken = "demo"

#下單張數
tradingVolume = 10

#----------------------------------
while True:  

    # 從 dealts API 取得最新兩筆成交明細數據 
    price_data = fugle_realtime_api('dealts',symbolId, apiToken)[:5]

    
    #策略條件：最新一筆成交量 大於等於 前一筆成交量的10倍 且為外盤成交
    if (price_data[0]['volume'] >= price_data[1]['volume']*10) and (price_data[0]['ask'] <= price_data[0]['price']):

        # 委託資訊
        order = OrderObject(
            apCode = APCode.Common,# 盤別
            buySell = Action.Buy,
            priceFlag= PriceFlag.LimitDown, # 跌停價買進
            price = None,
            stockNo = str(symbolId),
            quantity = tradingVolume,
        )

        # 送下單委託
        sdk.place_order(order)
        
        print('成功送出委託')
        print(f'買進張數：{tradingVolume} 張')

        break
        
    else:
        print(f"最新成交量 {str(price_data[0]['volume'])} 張")
        print(f'外盤成交：',str(price_data[0]['ask'] <= price_data[0]['price']))
        print('未達買進條件!')
        print('---')


成功送出委託
買進張數：10 張


In [12]:
# 查看委託回報
sdk.login()
orderResult = sdk.get_order_results()
print(orderResult[0])

{'market': 'T', 'workdate': '20220316', 'orddate': '20220315', 'ordtime': '162730367', 'syscode': 'FA', 'ordstatus': '1', 'ordno': '', 'preordno': 'P0B8666Z', 'stockno': '2884', 'buysell': 'B', 'apcode': '1', 'priceflag': '2', 'trade': '0', 'odprice': '0.0', 'orgqty': 10, 'matqty': 0, 'celqty': 0, 'celable': '1', 'errcode': '00000000', 'errmsg': '', 'avgPrice': '0', 'chgtime': '162730367', 'chgdate': '20220315', 'bsFlag': 'R', 'orgqty_share': 10000, 'matqty_share': 0, 'celqty_share': 0}


In [13]:
# 刪掉第 0 筆單
sdk.cancel_order(orderResult[0])

{'ordDate': '20220315',
 'ordTime': '162748728',
 'retcode': '000000',
 'retmsg': ''}